In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import plotly.express as px

In [ ]:
%run ../../modules/utils.ipynb
%run ../../modules/cds.ipynb
%run ../../modules/preprocessing.ipynb

In [ ]:
df3 = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/BaoLab/FirstBatch/Excels/S2to5combined_20ul_9min.xlsx')
df6 = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/BaoLab/FirstBatch/Excels/S6to10combined_20ul_9min.xlsx')
df3 = thermo_df(df3)
df6 = thermo_df(df6)
df3.shape, df6.shape

In [ ]:
df2 = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/BaoLab/FirstBatch/Excels/S2to5combined_20ul_40min.xlsx')
df7 = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/BaoLab/FirstBatch/Excels/S6to10combined_20ul_40min.xlsx')
df2 = thermo_df(df2)
df7 = thermo_df(df7)
df2.shape, df7.shape

In [ ]:
df2_25k = df2[(df2.Mass > 24000) & (df2.Mass < 24500)]
plotly_zone(df2_25k, y='Vol')

In [ ]:
df7_25k = df7[(df7.Mass > 24000) & (df7.Mass < 25000)]
plotly_zone(df7_25k, y='Vol')

In [ ]:
df2_big = df2[df2.Mass>22000].sort_values('Vol', ascending=False)[:20][['Mass', 'Vol', 'RT']]
df2_big # 2

In [ ]:
df7_big = df7[df7.Mass>22000].sort_values('Vol', ascending=False)[:10][['Mass', 'Vol', 'RT']]
df7_big # 0, 

In [ ]:

AminoAcid = 128.94
def handle_methyl(df, intact_mass_a, intact_mass_b):
    """ Given two intact Masses with Methyl difference, return ladders for both Masses
    """
    df_ms_1, _ = computational_data_seperation(df, df, full_mass=intact_mass_a, ignore_endpoints=True)
    df_ms_2, _ = computational_data_seperation(df, df, full_mass=intact_mass_b, ignore_endpoints=True)
    df_comm = df_ms_1[df_ms_1.Mass.isin(df_ms_2.Mass)]
#     plotly_zone(df_comm, title='Common Dots')
    df_ms_2_tmp = df_ms_2.copy()
    df_ms_2_tmp.Mass -= M
    df_comm_pre_methyl = match_dfs(df_ms_2_tmp, df_ms_1)
    
    df_ms_1_tmp = df_ms_1.copy()
    df_ms_1_tmp.Mass += M
    df_comm_post_methyl = match_dfs(df_ms_1_tmp, df_ms_2)

    df_A = pd.concat([df_comm, df_comm_pre_methyl])
#     plotly_zones(df_comm, df_comm_pre_methyl, title='Pre Methyl')
    df_B = pd.concat([df_comm, df_comm_post_methyl])
#     plotly_zones(df_comm, df_comm_post_methyl, title='Post Methyl')

    return df_A, df_B

In [ ]:
def comm_dots(df1, df2, intact_mass):
    df_ms_1, _ = computational_data_seperation(df1, df1, full_mass=intact_mass, ignore_endpoints=True)
    df_ms_2, _ = computational_data_seperation(df2, df2, full_mass=intact_mass, ignore_endpoints=True)
    df_comm = match_dfs(df_ms_2, df_ms_1)
    return df_comm

In [ ]:

masses = [24189.23, 24203.24, 24217.27, 24231.25]

In [ ]:
# common dots for methyled intact mass pairs
# masses = [24288.17982, 24302.1913, 24316.12684, 24330.1247, 24344.13365]
# masses = [24189.22295, 24203.25458, 24245.22615]
df2_A, df2_B = handle_methyl(df2, masses[0], masses[1])
df7_A, df7_B = handle_methyl(df7, masses[0], masses[1])
df2_A.shape, df2_B.shape, df7_A.shape, df7_B.shape

In [ ]:
df27_A, df27_B = handle_methyl(df_27, masses[0], masses[1])
df27_A.shape, df27_B.shape

In [ ]:
df_A = match_dfs(df7_A, df2_A)
plotly_zone(df_A)
df_B = match_dfs(df7_B, df2_B)
plotly_zone(df_B)
plotly_zones(df_A, df_B)

In [ ]:
df_mass2 = comm_dots(df2, df7, masses[0])
plotly_zone(df_mass2)

In [ ]:
df_mass7 = comm_dots(df7, df2, masses[0])
plotly_zone(df_mass7)

In [ ]:
idxs = list()
# create our callback function
def on_selection(trace, points, selector):
    print('points {}'.format(points.point_inds) )
    idxs.extend(points.point_inds)

def on_click(trace, points, selector):
#     print('points {}'.format(points.point_inds) )
    idxs.extend(points.point_inds)

df_sample = df_mass2
f = zone_selection(df_sample, on_selection=on_selection, on_click=on_click)
f

In [ ]:
df_chosen = df_sample.iloc[idxs]
df_zone = df_sample.drop(df_chosen.index)
df_zone.sort_values('Mass')

In [ ]:
df_chosen.shape, df_zone.shape
df_a = df_zone.copy()
df_b = df_chosen.copy()
df_a['is5p'] = True
df_b['is5p'] = False
df_merge = pd.concat([df_a, df_b])
df_merge.to_excel('/Users/bryan/Documents/BioPharmaFinder/Draw/SourceGlu/ladders_split_s3_24189.xlsx')

In [ ]:
df_merge = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/Draw/SourceGlu/ladders_split_s3_24189.xlsx')
df_merge.head()

In [ ]:
plotly_zones(df_merge[df_merge.is5p==True], df_merge[df_merge.is5p==False])

In [ ]:
df_src = df7
df_end_dot = df_src[(df_src.Mass>masses[0]-0.1) & (df_src.Mass<masses[0]+0.1)]
df_gap_dots = find_gap_dots(df_src, df_merge[df_merge.is5p==True], df_end_dot, orientation=5)

In [ ]:
df_zone = df_merge[df_merge.is5p==True]
plotly_zones(df_zone, df_gap_dots)
# plt.savefig('/Users/bryan/Documents/BioPharmaFinder/Draw/SourceGlu/masssum_s3_24189_5p.svg', dpi=300, transparent=transparent)

In [ ]:
df_a = df_zone.copy()
df_b = df_gap_dots.copy()
df_a['isSkeleton'] = True
df_b['isSkeleton'] = False
df_merge = pd.concat([df_a, df_b])
df_merge.to_excel('/Users/bryan/Documents/BioPharmaFinder/Draw/SourceGlu/masssum_s6_24189_5p.xlsx')

group = '24189'
full_mass = masses[0]
bcr = base_calling_random(df_merge, silence=True)
plot_basecalling(*bcr, annotate=False)
df_seats = process_mass_base_seats(df_merge, bcr[1], full_mass=full_mass)
df_seats.to_excel('/Users/bryan/Documents/BioPharmaFinder/Draw/SourceGlu/seats_s6_24189_5p.xlsx')

In [ ]:
bcr = base_calling_random(df_zone, silence=True)
plt, _ = plot_basecalling(*bcr)
plt.scatter(df_zone.Mass, df_zone.RT)

In [ ]:
# df_27 = pd.concat([df2, df7]).drop_duplicates()
df_27 = match_dfs(df7, df2)

In [ ]:
df_skeletons = list()
for idx in range(len(masses)):
    df_skeleton, _ = computational_data_seperation(df_27, df_27, full_mass=masses[idx], ignore_endpoints=True)
    df_skeletons.append(df_skeleton)

In [ ]:
df_skeleton_24217, _ = computational_data_seperation(df7, df7, full_mass=masses[2], ignore_endpoints=True)
plotly_zone(df_skeleton_24217)

In [ ]:
idxs = list()
# create our callback function
def on_selection(trace, points, selector):
    print('points {}'.format(points.point_inds) )
    idxs.extend(points.point_inds)

def on_click(trace, points, selector):
#     print('points {}'.format(points.point_inds) )
    idxs.extend(points.point_inds)

df_sample = df_skeleton_24217
f = zone_selection(df_sample, on_selection=on_selection, on_click=on_click)
f

In [ ]:
df_chosen = df_sample.iloc[idxs]
df_chosen.shape

In [ ]:
df_mid_3p = df_chosen.copy()
df_mid_3p.sort_values('Mass')

In [ ]:
df_mid = df_chosen.copy()
df_mid_core = filter_dots_in_mid_gap(df_mid)
df_mid_core.shape

In [ ]:
plotly_zones(df_mid, df_mid_core)
plotly_zone(df_mid_core)

In [ ]:
components(5541.719 - 4571.528)

In [ ]:
plotly_multi_zones(df_skeletons[2:3])
masses[2]

In [ ]:
# common dots in sample2 and sample7
masses = [24189.23, 24203.24, 24217.27, 24231.25]
# masses = [24288.17982, 24302.1913, 24316.12684, 24330.1247, 24344.13365]
# masses = [24189.22295, 24203.25458, 24245.22615]

AminoAcid = 128.94
df_comms = list()
for mass in masses:
    df_ms_1, _ = computational_data_seperation(df2, df2, full_mass=mass, ignore_endpoints=True)
    df_ms_2, _ = computational_data_seperation(df7, df7, full_mass=mass, ignore_endpoints=True)
    df_comm = match_dfs(df_ms_1, df_ms_2)
    df_comms.append(df_comm)
#     plotly_zone(df_ms_1)
#     plotly_zone(df_ms_2)
    plotly_zone(df_comm)
    

# 24189 Sample2

In [ ]:
plotly_zone(df_comms[0])

In [ ]:
idxs = list()
# create our callback function
def on_selection(trace, points, selector):
    print('points {}'.format(points.point_inds) )
    idxs.extend(points.point_inds)

def on_click(trace, points, selector):
#     print('points {}'.format(points.point_inds) )
    idxs.extend(points.point_inds)

df_sample = df_comms[0]
f = zone_selection(df_sample, on_selection=on_selection, on_click=on_click)
f

In [ ]:
df_chosen = df_sample.iloc[idxs]
df_chosen.shape

In [ ]:
df_24189 = df_sample.drop(df_chosen.index).copy()
df_24189.shape

In [ ]:
df_sample = df_comms[0]
df_24189_3p = df_sample.drop(df_24189.index)
plotly_zones(df_24189, df_24189_3p)

In [ ]:
df_24189['is5p'] = True
df_24189_3p['is5p'] = False
df_merge_24189 = pd.concat([df_24189, df_24189_3p])

In [ ]:
# df_merge_24189.to_excel('/Users/bryan/Documents/BioPharmaFinder/BaoLab/MassSum/GraphsOutput/24189')
plt, fig = local_plot_zones(df_24189, df_24189_3p)
output(df_merge_24189, plt, fig, 1, 'mass_sum')

In [ ]:
plotly_zone(df_24189)

In [ ]:
full_mass = masses[0]
fullmass_dot = df2[(df2.Mass>full_mass-0.1) & (df2.Mass<full_mass+0.1)]
df_gap_dots = find_gap_dots(df2, df_24189, fullmass_dot, orientation=5)
df_gap_dots.shape

In [ ]:
plotly_zones(df_24189, df_gap_dots)
# plt = plot_zones(df_skeleton, df_gap_dots)
# plt.savefig('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/MassSum/II/Phe_MFE0724/{}/{}_5p.png'.format(group, int(full_mass)), transparent=True)

In [ ]:
df_merge = pd.concat([df_24189, df_gap_dots]).drop_duplicates()


In [ ]:
idxs = list()
# create our callback function
def on_selection(trace, points, selector):
    print('points {}'.format(points.point_inds) )
    idxs.extend(points.point_inds)

def on_click(trace, points, selector):
#     print('points {}'.format(points.point_inds) )
    idxs.extend(points.point_inds)

df_sample = df_merge
f = zone_selection(df_sample, on_selection=on_selection, on_click=on_click)
f

In [ ]:
df_chosen = df_sample.iloc[idxs]
df_chosen.shape

In [ ]:
plotly_zone(df_chosen)

In [ ]:
bcr = base_calling_random(df_chosen, silence=True)
plot_basecalling(*bcr, annotate=False)

In [ ]:
group = 'Group_24189'
df_seats = process_mass_base_seats(df_merge, bcr[1], full_mass=full_mass)
df_seats.to_excel('/Users/bryan/Documents/BioPharmaFinder/BaoLab/MassSum/FirstBatch/{}/seats_{}_5p.xlsx'.format(group, int(full_mass)))
df_merge.to_excel('/Users/bryan/Documents/BioPharmaFinder/BaoLab/MassSum/FirstBatch/{}/{}_5p.xlsx'.format(group, int(full_mass)))

# 24189 Sample7

In [ ]:
%run ../../modules/utils.ipynb

In [ ]:
full_mass = masses[0]
fullmass_dot_7 = df7[(df7.Mass>full_mass-0.1) & (df7.Mass<full_mass+0.1)]
df_gap_dots_7 = find_gap_dots(df7, df_24189, fullmass_dot, orientation=5)
df_gap_dots_7.shape

In [ ]:
df_mid = dots_in_mid_gap(df7, 13555.735066273, 14205.8059294037)
# df_mid = df_mid.sort_values('Mass', ascending=False)
df_mid_core = filter_dots_in_mid_gap(df_mid)
df_mid_core

In [ ]:
plotly_zones(df_24189, df_gap_dots_7[df_gap_dots_7.RT < 30])

In [ ]:
df_24189.head()

In [ ]:
df_m = pd.concat([df_24189[['Mass', 'RT', 'Vol']], df_gap_dots_7[df_gap_dots_7.RT < 30]]).drop_duplicates()
bcr = base_calling_random(df_m, silence=True)
df_seats = process_mass_base_seats(df_m, bcr[1], full_mass=masses[0])
df_seats.to_excel('/Users/bryan/Documents/BioPharmaFinder/BaoLab/MassSum/GraphsOutput/seats_{}_5p_core.xlsx'.format(int(full_mass)))

In [ ]:
# df_mid = dots_in_mid_gap(df7, 12600.63, 13555.74)
df_mid = dots_in_mid_gap(df7, 5912.7, 8489.055)
df_mid = df_mid.sort_values('Mass', ascending=False)
df_mid_core = filter_dots_in_mid_gap(df_mid)
df_mid_core

In [ ]:
df_mid

In [ ]:
df_x = df_mid.Mass.copy()
df_y = df_mid.Mass.copy()
np_x = np.array(df_x)
np_y = np.array(df_y)

In [ ]:
np_xy = np_x[:, np.newaxis] - np_y
np_xy

In [ ]:

def soft_connected(mass_diff):
    if mass_diff <= 0:
        return False
    df = components(mass_diff)
    return not df.empty
comp_vec = np.vectorize(soft_connected)

In [ ]:
np_conns = comp_vec(np_xy)
df_conns = pd.DataFrame(np_conns)
df_conns

In [ ]:
idx_pairs = list(df_conns[df_conns == True].stack().index)
print(idx_pairs)
# df_mid.iloc[idx_pairs[0][0]], df_mid.iloc[idx_pairs[0][1]]

In [ ]:
def zip_list(G):
    node_count_list = [(node, len(G.edges(node))) for node in G.nodes]
    node_count_list.sort(key=lambda x: x[1])
    return node_count_list

In [ ]:
import networkx as nx
G = nx.Graph()
for pair in idx_pairs:
    a = df_mid.iloc[pair[0]].Mass
    b = df_mid.iloc[pair[1]].Mass
#     if pair[0] == 1:
#         continue
    G.add_edge(a, b)

In [ ]:

while True:
    l = zip_list(G)
    print(l)
    max_edges = l[-1][1]
    if l[0][1] >= max_edges:
        break
    G.remove_node(l[0][0])
l = zip_list(G)
print('final: ', l)

In [ ]:
def soft_connected(mass_diff):
    if mass_diff <= 0:
        return False
    df = components(mass_diff)
    return not df.empty
comp_vec = np.vectorize(soft_connected)

def zip_list(G):
    node_count_list = [(node, len(G.edges(node))) for node in G.nodes]
    node_count_list.sort(key=lambda x: x[1])
    return node_count_list

import networkx as nx
def filter_dots_in_mid_gap(df):
    df_x = df.Mass.copy()
    df_y = df.Mass.copy()
    np_x = np.array(df_x)
    np_y = np.array(df_y)
    np_xy = np_x[:, np.newaxis] - np_y
#     np_xy

    np_conns = comp_vec(np_xy)
    df_conns = pd.DataFrame(np_conns)
#     df_conns

    idx_pairs = list(df_conns[df_conns == True].stack().index)
#     print(idx_pairs)
    # df_mid.iloc[idx_pairs[0][0]], df_mid.iloc[idx_pairs[0][1]]

    G = nx.Graph()
    for pair in idx_pairs:
        a = df_mid.iloc[pair[0]].Mass
        b = df_mid.iloc[pair[1]].Mass
    #     if pair[0] == 1:
    #         continue
        G.add_edge(a, b)

    while True:
        l = zip_list(G)
#         print(l)
        max_edges = l[-1][1]
        if l[0][1] >= max_edges:
            break
        G.remove_node(l[0][0])
    l = zip_list(G)
#     print('final: ', l)
    masses = [node[0] for node in l]
    print('reserved masses: ', masses)
    return df[df.Mass.isin(masses)]

In [ ]:
%run ../../modules/utils.ipynb
df_mid_core = filter_dots_in_mid_gap(df_mid)
df_mid_core

In [ ]:
# scc = nx.kosaraju_strongly_connected_components(G)
# nx.is_strongly_connected(G)
scc = nx.connected_components(G)

In [ ]:
list(scc)

In [ ]:
df_mid

In [ ]:
df_comp = components(13555.74 - 12600.63)
not df_comp.empty

In [ ]:
df_a = df_24189.copy()
df_b = df_gap_dots_7[df_gap_dots_7.RT < 30].copy()
df_a['isSkeleton'] = True
df_b['isSkeleton'] = False
df_m = pd.concat([df_a, df_b])
plt, fig = local_plot_zones(df_a, df_b)
output(df_m, plt, fig, 2, 'gap_fill')

In [ ]:
df_m = df_m[['Mass', 'RT', 'Vol']].drop_duplicates()

In [ ]:
%run ../../modules/cds.ipynb
full_mass=masses[0]
bcr = base_calling_random(df_m, silence=True)
df_seats = process_mass_base_seats(df_m, bcr[1], full_mass=masses[0])
df_seats.to_excel('/Users/bryan/Documents/BioPharmaFinder/BaoLab/MassSum/GraphsOutput/seats_{}_5p_core.xlsx'.format(int(full_mass)))

In [ ]:
def local_plot_zone(df, trend=False, order=1, y='RT'):
    fig = plt.figure(figsize=(8.5, 4))
    plt.xlabel('Monoisotopic Mass (Da)', fontname="Arial", fontsize=15, color='black')
    if y == 'RT':
        plt.ylabel('Retention Time (min)', fontname="Arial", fontsize=15, color='black')
    else:
        plt.ylabel('Intensity', fontname="Arial", fontsize=15, color='black')
    plt.xticks(fontname="Arial", size=13, color='black')
    plt.yticks(fontname="Arial", size=13, color='black')
    if trend:
        sns.regplot(df.Mass, df[y], order=order)
    else:
        plt.scatter(df.Mass, df[y])
    
    return plt, fig

def local_plot_zones(df3p, df5p, trend=False, y='RT'):
    fig = plt.figure(figsize=(8.5, 4))
    plt.xlabel('Monoisotopic Mass (Da)', fontname="Arial", fontsize=15, color='black')
    plt.ylabel('Retention Time (min)', fontname="Arial", fontsize=15, color='black')
    plt.xticks(fontname="Arial", size=13, color='black')
    plt.yticks(fontname="Arial", size=13, color='black')
    if trend:
        sns.regplot(df3p.Mass, df3p[y])
        sns.regplot(df5p.Mass, df5p[y], order=2)
    else:
        plt.scatter(df3p.Mass, df3p[y])
        plt.scatter(df5p.Mass, df5p[y])
    
    return plt, fig

def output(df, plt, fig, idx, svg_name):
    fig.tight_layout()
    plt.savefig(f'/Users/bryan/Documents/BioPharmaFinder/BaoLab/MassSum/GraphsOutput/Fig{idx}_{svg_name}.svg', dpi=300, transparent=True)
    df.to_excel(f'/Users/bryan/Documents/BioPharmaFinder/BaoLab/MassSum/GraphsOutput/S{idx}_{svg_name}.xlsx')

In [ ]:
components(5567-4916)

In [ ]:
df_merge_7 = pd.concat([df_24189, df_gap_dots_7]).drop_duplicates()

In [ ]:
df_merge_7 = df_merge_7[df_merge_7.RT < 30].copy()

In [ ]:
idxs = list()
# create our callback function
def on_selection(trace, points, selector):
    print('points {}'.format(points.point_inds) )
    idxs.extend(points.point_inds)

def on_click(trace, points, selector):
#     print('points {}'.format(points.point_inds) )
    idxs.extend(points.point_inds)

df_sample = df_merge_7
f = zone_selection(df_sample, on_selection=on_selection, on_click=on_click)
f

In [ ]:
df_chosen = df_sample.iloc[idxs]
df_chosen.shape

In [ ]:
plotly_zone(df_chosen)

In [ ]:
bcr7 = base_calling_random(df_chosen, silence=True)

In [ ]:
group = 'Group_24189'
df_seats = process_mass_base_seats(df_merge_7, bcr7[1], full_mass=full_mass)
df_seats.to_excel('/Users/bryan/Documents/BioPharmaFinder/BaoLab/MassSum/FirstBatch/{}_RSV/seats_{}_5p.xlsx'.format(group, int(full_mass)))
df_merge.to_excel('/Users/bryan/Documents/BioPharmaFinder/BaoLab/MassSum/FirstBatch/{}_RSV/{}_5p.xlsx'.format(group, int(full_mass)))

# 24203 Sample 7

In [ ]:
idxs = list()
# create our callback function
def on_selection(trace, points, selector):
    print('points {}'.format(points.point_inds) )
    idxs.extend(points.point_inds)

def on_click(trace, points, selector):
#     print('points {}'.format(points.point_inds) )
    idxs.extend(points.point_inds)

df_sample = df_comms[1]
f = zone_selection(df_sample, on_selection=on_selection, on_click=on_click)
f

In [ ]:
df_comp1 = components(12945.64 - 11661.46)

In [ ]:
12945.64 - (2*G + U + U + 2)

In [ ]:
components(12945.64 - 10682.4)

In [ ]:
components(12945.64 - 11641.4946)

In [ ]:
components(11641.4946 - 10682.4)

In [ ]:
components(12945.64 - 11661.46)

In [ ]:
components(11661.46 - 10682.4)

In [ ]:
df_chosen_24203 = df_sample.iloc[idxs]
df_chosen_24203.shape

In [ ]:
df_24203 = df_sample.drop(df_chosen_24203.index).copy()
df_24203.shape

In [ ]:
%run ../../modules/preprocessing.ipynb

In [ ]:
df_c = match_dfs(df2, df7)
df_c.shape

In [ ]:
plotly_zone(df_c)

In [ ]:
df_c_main = df_c[df_c.Mass > 2900]

In [ ]:
bcr = base_calling_random(df_c_main, silence=True)
plot_basecalling(*bcr, annotate=False)
len(bcr[1])

In [ ]:
print(len([i for i in bcr[1] if i[2] == 'mnm5s2U']),
len([i for i in bcr[1] if i[2] == 'X']),
len([i for i in bcr[1] if i[2] == 'T']),
len([i for i in bcr[1] if i[2] == 'g']),
len([i for i in bcr[1] if i[2] == 's4U']),
len([i for i in bcr[1] if i[2] == 'ms2io6A']))

In [ ]:
%run ../../modules/utils.ipynb

In [ ]:
full_mass = masses[1]
fullmass_dot_24203 = df7[(df7.Mass>full_mass-0.1) & (df7.Mass<full_mass+0.1)]
df_24203_gap_dots = find_gap_dots(df7, df_24203, fullmass_dot_24203, orientation=5, major=True)
df_24203_gap_dots.shape

In [ ]:
plotly_zones(df_24203, df_24203_gap_dots)

In [ ]:
dots_in_mid_gap()

In [ ]:
# masses = [24288.17982, 24302.1913, 24316.12684, 24330.1247, 24344.13365] 
# masses = [24189.22295, 24203.25458, 24245.22615]
masses = [24189.23, 24203.24, 24217.27, 24231.25]

df_ms_list = list()
df_common = pd.DataFrame()
for idx in range(1, len(masses)):
    df_ms_1, _ = computational_data_seperation(df2, df2, full_mass=masses[idx-1], ignore_endpoints=True)
    df_ms_2, _ = computational_data_seperation(df2, df2, full_mass=masses[idx], ignore_endpoints=True)
    df_comm = df_ms_1[df_ms_1.Mass.isin(df_ms_2.Mass)]
    plotly_zone(df_comm)

In [ ]:
# full_mass = df_big.iloc[0].Mass
# full_mass = 23277.1 #23281.1 #24206.2 #24330.2261
# full_mass = 23595.15 #24234.26
full_mass = 24203.26029 #24189.22863 #24330.1247 #24201.18331 #24330.1247 #24302.19227 #23057.09
print(full_mass)
df_ms_3, df_ms_5  = computational_data_seperation(df7, df7, full_mass=full_mass, ignore_endpoints=True)
# df_ms.shape
plotly_zones(df_ms_3, df_ms_5)

In [ ]:
df_ms_bcr, mass_pairs = base_calling_random(df_ms)
df_ms_bcr.shape, len(mass_pairs)
plot_basecalling(df_ms_bcr, mass_pairs, annotate=False)

In [ ]:
mass_3p = [pair[0] for pair in mass_pairs]
mass_5p = [pair[1] for pair in mass_pairs]
mass_3p5p = set(mass_3p) & set(mass_3p)
mass_3ponly = set(mass_3p) - set(mass_5p)
mass_5ponly = set(mass_5p) - set(mass_3p)
mass_else = mass_3ponly | mass_5ponly

df_mass_3p5p = df_ms_bcr[df_ms_bcr.Mass.isin(mass_3p5p)]
# plot_zone(df_mass_3p5p)
# plot_zone(df_ms_bcr[df_ms_bcr.Mass.isin(mass_else)])
plt.figure(figsize=(12, 9))
plt.xlabel('Mass')
plt.ylabel('RT')
plt.scatter(df_mass_3p5p.Mass, df_mass_3p5p.RT)
len(mass_3p5p), len(mass_else)
# plt.savefig('/Users/bryan/Documents/BioPharmaFinder/BaoLab/FirstBatch/Excels/S6to10combined_20ul_40min_skeleton_1.png', transparent=True)

#    Rank to get the full mass value

In [ ]:
df_glu = df7[df7.Mass > 1500]

In [ ]:
df_glu = df2[df2.Mass > 2329]

In [ ]:
df3p_mass_np = np.array(df_glu['Mass'])
df5p_mass_np = np.array(df_glu['Mass'])
mass_sum_np = df3p_mass_np[:, np.newaxis] + df5p_mass_np
mass_sum_0 = np.round(mass_sum_np, 0)
mass_sum_1 = np.round(mass_sum_np, 1)
mass_sum_2 = np.round(mass_sum_np, 2)
mass_sum_3 = np.round(mass_sum_np, 3)

In [ ]:
res1 = np.unique(mass_sum_1, return_counts=True)
res2 = np.unique(mass_sum_2, return_counts=True)
res3 = np.unique(mass_sum_3, return_counts=True)

In [ ]:
res1_s0 = pd.Series(res1[0])
res1_s1 = pd.Series(res1[1])

res2_s0 = pd.Series(res2[0])
res2_s1 = pd.Series(res2[1])

res3_s0 = pd.Series(res3[0])
res3_s1 = pd.Series(res3[1])

In [ ]:
res1_s1_max = np.max(res1_s1)
res1_s2_max = np.max(res2_s1)
res1_s3_max = np.max(res3_s1)
print("max s1 {} s2 {} s3 {}".format(res1_s1_max, res1_s2_max, res1_s3_max))

In [ ]:
res1_idxs = res1_s1[res1_s1 > 400].index
res1_s0[res1_idxs]
l = list(zip(res1_s0[res1_idxs], res1_s1[res1_idxs]))
[i for i in l if i[0] > 23000]

In [ ]:
res2_idxs = res2_s1[res2_s1 > 80].index
l = list(zip(res2_s0[res2_idxs], res2_s1[res2_idxs]))
[i for i in l if i[0] > 23000]

In [ ]:
res3_idxs = res3_s1[res3_s1 > 10].index
l = list(zip(res3_s0[res3_idxs], res3_s1[res3_idxs]))

# Top vols

In [ ]:
plotly_zone(df_glu)

In [ ]:
df_glu = df_glu.sort_values('Vol', ascending=False)

In [ ]:
df_top = df_glu[(df_glu.Mass > 11000) & (df_glu.Mass < 20000)][:500]

In [ ]:
plotly_zone(df_top)

# Glu DataSet 7

In [ ]:
import plotly.graph_objects as go

idxs = list()
# create our callback function
def update_point(trace, points, selector):
    print('points {}'.format(points.point_inds) )
    idxs.extend(points.point_inds)

df_draw = df7[(df7.Mass > 5000) & (df7.Mass < 22000)]
# df_draw = df7_chosen[:100] #df7
x = df_draw.Mass
y = df_draw.RT
fig = go.Scatter(x=x, y=y, mode='markers')
# fig.update_layout(width=1024, height=768)
f = go.FigureWidget(fig)
scatter = f.data[0]
scatter.on_selection(update_point)
f

In [ ]:
df7_1 = df_draw.iloc[idxs]
df7_1.info()

In [ ]:
bcr_5p = base_calling_random(df7_1[:100])
plot_basecalling(*bcr_5p, annotate=False)

In [ ]:
df7_chosen = df_draw.iloc[idxs]
df7_chosen.info()

In [ ]:
df7_chosen = df7_chosen.sort_values('Vol', ascending=False)

In [ ]:
bcr_5p = base_calling_random(df7_chosen[:100])
plot_basecalling(*bcr_5p, annotate=True)

# Glu Dataset 2

In [ ]:
df2_top = df2[(df2.Mass > 2300) & (df2.Mass < 22000)].sort_values('Vol').groupby('RT').tail(1)

In [ ]:
import plotly.graph_objects as go

idxs = list()
# create our callback function
def update_point(trace, points, selector):
    print('points {}'.format(points.point_inds) )
    idxs.extend(points.point_inds)

# df_draw = df2[(df2.Mass > 12329) & (df2.Mass < 22000) & (df2.RT < 25) & (df2.RT > 8)]
# df_draw = df2_chosen[:100] #df7
df_draw = df2_top[(df2_top.RT < 25) & (df2_top.RT > 8)]
x = df_draw.Mass
y = df_draw.RT
fig = go.Scatter(x=x, y=y, mode='markers')
f = go.FigureWidget(fig)
f.update_layout(autosize=False, width=1024, height=700, paper_bgcolor="LightSteelBlue",
    margin=dict(
        l=20,
        r=20,
        b=20,
        t=20,
        pad=4
    ))
scatter = f.data[0]
scatter.on_selection(update_point)
f

In [ ]:
df2_chosen = df_draw.iloc[idxs]
df2_chosen.info()

In [ ]:
df2_chosen = df2_chosen.sort_values('Vol', ascending=False)

In [ ]:
plot_zone(df2_chosen[:100])

In [ ]:
bcr_5p = base_calling_random(df2_chosen[:100])
plot_basecalling(*bcr_5p, annotate=True)

In [ ]:
# df2_chosen.to_excel('/Users/bryan/Documents/BioPharmaFinder/BaoLab/Excels/S2to5combined_20ul_40min_skeleton_1.xlsx')
df2_chosen = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/BaoLab/Excels/S2to5combined_20ul_40min_skeleton_1.xlsx')

In [ ]:
df2_chosen = df2_chosen.sort_values('Mass', ascending=True)
df2_chosen.tail()

In [ ]:
df2_exclude_h2o = df2_chosen
df2_exclude_h2o.Mass = df2_exclude_h2o.Mass - 18.0106

In [ ]:
df2_exclude_h2o_restricted = df2_exclude_h2o[(df2_exclude_h2o.Mass > 8000) & (df2_exclude_h2o.Mass < 11000)]
df2_exclude_h2o_restricted.to_excel('/Users/bryan/Documents/BioPharmaFinder/BaoLab/Excels/S2to5combined_20ul_40min_skeleton_h2o_8k_11k.xlsx')

In [ ]:
df2_chosen_restricted = df2_chosen[(df2_chosen.Mass > 8000) & (df2_chosen.Mass < 11000)]
df2_chosen_restricted.to_excel('/Users/bryan/Documents/BioPharmaFinder/BaoLab/Excels/S2to5combined_20ul_40min_skeleton_8k_11k.xlsx')